<a href="https://colab.research.google.com/github/ncn-foreigners/job-ads-classifier/blob/main/job_classification_examples_polski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab and environment settings

Printing in colab

In [1]:
from google.colab.data_table import DataTable
DataTable.max_columns = 60

Clone repository

In [2]:
!git clone https://github.com/ncn-foreigners/job-ads-classifier.git 

Cloning into 'job-ads-classifier'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 139 (delta 54), reused 60 (delta 11), pack-reused 0
Receiving objects: 100% (139/139), 3.25 MiB | 11.08 MiB/s, done.
Resolving deltas: 100% (54/54), done.


Set-up virtual env

In [3]:
!pip install virtualenv
!virtualenv classifier
!source /content/classifier/bin/activate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 4.1 MB/s 
     |████████████████████████████████| 468 kB 45.6 MB/s 
created virtual environment CPython3.7.15.final.0-64 in 1013ms
  creator CPython3Posix(dest=/content/classifier, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.2.2, setuptools==65.3.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


Upgrade pip

In [4]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pip-22.3-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


Download models

In [ ]:
!gdown 126YUU3X9AfzdtEJgANzusgo7msonoCzu
!unzip models-v1.1.zip
!rm -rf __MACOSX

In [6]:
!mkdir ./job-ads-classifier/models
!mv transformer_model ./job-ads-classifier/models/
!mv linear_model ./job-ads-classifier/models/

Install required packages

In [7]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install -r /content/job-ads-classifier/requirements-py37-colab.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.5/541.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.7/361.7 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.1/582.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 40.9 MB/s eta 0:00:00
  

In [ ]:
exit()

## Working with trained models

In [1]:
classifier_dir="/content/job-ads-classifier"
models_dir="/content/job-ads-classifier/models"

Load required modules

In [2]:
import sys
sys.path.insert(1, '/content/job-ads-classifier')

In [3]:
import numpy as np
import pandas as pd
from job_offers_classifier.load_save import load_to_df, save_obj, load_obj
from job_offers_classifier.job_offers_classfier import *
import torch
from torchmetrics import MetricCollection, Recall, Accuracy, ConfusionMatrix
execfile(f"{classifier_dir}/functions.py")

Example data -- job offers from Public Sercice

In [4]:
kprm_df = pd.read_xml(f"{classifier_dir}/example-data/kprm-2022-10-21.xml", xpath = "./new/oferta")
kprm_df = kprm_df.replace("\\<\\!\\[CDATA\\[ ", "", regex=True)
kprm_df = kprm_df.replace(" \\]\\]\\>", "", regex=True)
kprm_df = kprm_df.replace(r'<[^<>]*>', ' ', regex=True)
kprm_df = kprm_df.replace('&nbsp;', ' ', regex=True)
kprm_df = kprm_df.replace('&dash;', ' ', regex=True)
kprm_df = kprm_df.replace('&oacute;', ' ', regex=True)
kprm_df = kprm_df.replace('\r', ' ', regex=True)
kprm_df['desc'] = kprm_df[['stanowisko', 'do_spraw', 'nazwa_firmy', 'poledodatkowelista1', 'poledodatkowelista3']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
kprm_df[["unid", "nazwa_firmy", "stanowisko", "do_spraw", "poledodatkowelista1", "poledodatkowelista3"]].head(n=2)

,unid,nazwa_firmy,stanowisko,do_spraw,poledodatkowelista1,poledodatkowelista3
0,108903,Główny Urząd Miar w Warszawie,metrolog,pomiarów w dziedzinie akustyki w Świętokrzyski...,"bierze udział w pracach badawczych, rozwojow...","* MIEJSCE WYKONYWANIA PRACY Warszawa, u..."
1,108902,Ministerstwo Rodziny i Polityki Społecznej w W...,referendarz,współpracy prawnej FGŚP,­Bierze udział w pracach legislacyjnych doty...,Praca administracyjno-biurowa. Opracowy...


In [5]:
kprm_df.desc[0]

'metrolog pomiarów w dziedzinie akustyki w Świętokrzyskim Kampusie Laboratoryjnym Głównego Urzędu Miar (ŚKLGUM) Główny Urząd Miar w Warszawie   bierze udział w pracach badawczych, rozwojowych i technicznych związanych z tworzeniem nowych  i doskonaleniem istniejących rozwiązań w zakresie wzorców jednostek miar, metod i stanowisk pomiarowych,  automatyzacji stanowisk pomiarowych w celu potwierdzenia możliwości pomiarowych ŚKLGUM  bierze udział w przygotowaniu i uruchomieniu stanowisk pomiarowych w ŚKLGUM w Kielcach, w tym  w szczególności w opracowywaniu dokumentacji technicznej dla nowotworzonego zespołu stanowisk  pomiarowych z komorą bezechową dużą i małą, udział w szkoleniach w zakresie obsługi aparatury pomiarowej na ww.  stanowiskach, pogłębianie wiedzy specjalistycznej z zakresu obsługi budowanych stanowisk pomiarowych  w celu nabycia zaawansowanej wiedzy i umiejętności niezbędnych do profesjonalnego wykonywania  obowiązków w ŚKLGUM  wykonuje prace związane z utrzymywaniem i dosk

### Logistic regression model using tf-df matrix

In [6]:
model_paper_linear = LinearJobOffersClassifier()
model_paper_linear.load(f"{models_dir}/linear_model")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [7]:
kprm_df_zaw1 = model_paper_linear.predict(
    X_text=list(kprm_df["desc"]),
    output_level=0,
    format="dataframe"
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:02<00:00, 174.40it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1611.32it/s]


Distribution of occupancy codes

In [8]:
np.sum(kprm_df_zaw1, axis=0)

0      0.017761
1     13.853405
2    254.529663
3    111.025528
4     53.315598
5      6.214397
6      0.951644
7      3.155583
8      2.712708
9      4.203103
dtype: float32

Top 5 occupancy codes

In [9]:
kprm_df_zaw6_top5 = model_paper_linear.predict(
    X_text=list(kprm_df["desc"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:05<00:00, 89.28it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1618.94it/s]


In [10]:
kprm_df_zaw6_top5.head(n=5)

,class_1,class_2,class_3,class_4,class_5,prob_1,prob_2,prob_3,prob_4,prob_5
0,214921,214922,214990,214903,214932,0.635320,0.203939,0.019870,0.012856,0.009343
1,242217,242190,242102,242290,242232,0.818859,0.063468,0.017318,0.011158,0.006835
2,242217,241107,241190,241103,241102,0.697043,0.109484,0.087459,0.032341,0.012461
3,242217,711101,335290,411003,334306,0.271974,0.106116,0.072340,0.069055,0.061913
4,325512,335990,325502,335905,334306,0.269934,0.133360,0.103502,0.067749,0.048600


### Transformer model

In [11]:
model_paper_trans = TransformerJobOffersClassifier(gpus=0, threads=2) ## bez gpu
#model_paper_trans = TransformerJobOffersClassifier(gpus=1, threads=2)
model_paper_trans.load(f"{models_dir}/transformer_model")

Initializing TransformerClassifier with model_name=allegro/herbert-base-cased, output_type=linear, num_labels=2904, learning_rate=1e-05, weight_decay=0.01, warmup_steps=50 ...


Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.sso.sso_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
kprm_df_zaw1_trans = model_paper_trans.predict(
    X=list(kprm_df["desc"]),
    output_level=0,
    format="dataframe"
)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/886k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

...

Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


TypeError: ignored

Distribution of occupancy codes

In [ ]:
np.sum(kprm_df_zaw1_trans, axis=0)

NameError: ignored

In [ ]:
kprm_df_zaw6_top5_trans = model_paper_trans.predict(
    X=list(kprm_df["desc"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

In [13]:
kprm_df_zaw6_top5_trans.head(n=5)

NameError: ignored

Compare linear vs transformer

In [ ]:
kprm_df["lin"] = kprm_df_zaw6_top5["class_1"]
kprm_df["trans"] = kprm_df_zaw6_top5_trans["class_1"]
kprm_df[kprm_df["lin"].str.slice(0,1) != kprm_df["trans"].str.slice(0,1)][
    ["trans", "lin", "stanowisko", "do_spraw", "desc"]].head(n=5)

,trans,lin,stanowisko,do_spraw,desc
3,242307,441590,referent,ds. kadr i płac,referent ds. kadr i płac Powiatowy Inspektorat...
12,242204,132102,audytor wewnętrzny,None,audytor wewnętrzny None Ministerstwo Spraw Zag...
18,411003,341103,podreferendarz,obsługi kancelaryjnej,podreferendarz obsługi kancelaryjnej Ministers...
21,311104,216590,inspektor wojewódzki,państwowego zasobu geodezyjnego i kartograficz...,inspektor wojewódzki państwowego zasobu geodez...
25,411003,242217,referent,sekretariatu,referent sekretariatu Izba Administracji Skarb...
